<a href="https://colab.research.google.com/github/hamagami/preprocess/blob/main/06_02_water.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggleの水質に関するデータの例　ほぼ数値データ

In [ ]:
!pip install -U  pandas_profiling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pdp

ph(ph値)　　Hardness(硬度) Solid(溶解固形物)　Chlormines(クロラミン) Sulfate(硫酸塩)　Conductivity(導電率) 	Organic_carbon(有機炭素）	Trihalomethanes(トリハロメタン)	Turbidity(濁度) 	Potability(飲用可能性 1 OK  0 NG）

In [ ]:
df = pd.read_csv("https://dl.dropbox.com/s/g8htb1fmd3uo24a/water_potability.csv")
df.head()

## 欠損値，統計量のチェック

In [ ]:
df.info()

In [ ]:
df.describe()

ヒストグラムのチェック

In [ ]:
for col in df.columns:
  plt.hist(df[col].values)
  plt.title(col)
  plt.show()

## 欠損値を正確に把握

In [ ]:
df.isnull().sum()

データは結構数あるので，NaNはドロップする

In [ ]:
df= df.dropna()
df.info()

Potability のバランス　を確認



In [ ]:
df.Potability.value_counts() #１のほうが少ない

データのリサンプリングによって調整

In [10]:
zero  = df[df['Potability']==0]   #Potability 0 (飲めない)のセット
one = df[df['Potability']==1]  #Potability 1(飲める）のセット
from sklearn.utils import resample

df_minority_upsampled = resample(one, replace = True, n_samples = 1200) 
#concatenate
df = pd.concat([zero, df_minority_upsampled])

from sklearn.utils import shuffle
df = shuffle(df) # shuffling so that there is particular sequence

In [ ]:
df.Potability.value_counts() #０と１の数が同じになる

In [ ]:
pdp.ProfileReport(df)

Portabilityとの相関を数値で確認

In [ ]:
df.corr().abs()['Potability'].sort_values(ascending = False)

In [14]:
X = df.drop(['Potability'], axis = 1)
y = df['Potability']

## 説明変数の標準化

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features= X.columns
X[features] = sc.fit_transform(X[features])

## 判別器のモジュール

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

訓練データとテストデータに分割

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)

# 以下，いくつかの判別機械学習アルゴリズムをつかって学習　
ただしハイパーパラメータの調整は行っていないので厳密な性能評価ではない

## ロジスティック回帰

In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

## ニューラルネットワーク

In [ ]:
clf = MLPClassifier(solver="adam",random_state=0, max_iter=10000)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

## SVC (SVMによるクラス判別）

In [ ]:
clf = SVC(gamma="scale")
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

## ランダムフォレスト

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, random_state=42)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

## AdaBoost

In [ ]:
ada = AdaBoostClassifier(n_estimators= 600, learning_rate= 1 )
ada.fit(X_train,y_train)
y_pred = ada.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))


## XGBoost


In [ ]:
xgb = XGBClassifier(n_estimators= 250, learning_rate= 0.8)
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))
